In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold

In [2]:
falseDF=pd.read_csv('falseDF.csv')
falseDF

,id,Age,Annual_Premium,Vintage,Response,Response_y,Response_y.1,Response_y.2,Response_y.3,Response_y.4,Response_y.5,Response_y.6
0,0,0.247059,0.120521,0.625418,0.0,0.139688,0.177725,0.240530,0.123132,0.228493,0.121274,0.192496
1,2,0.294118,0.070428,0.849498,0.0,0.103297,0.041404,0.004198,0.123132,0.000638,0.085700,0.026222
2,3,0.411765,0.004869,0.254181,0.0,0.103297,0.177725,0.240530,0.123132,0.228493,0.102073,0.204927
3,4,0.423529,0.059150,0.983278,0.0,0.103297,0.177725,0.004198,0.123132,0.000638,0.067435,0.026222
4,5,0.364706,0.052114,0.658863,0.0,0.103297,0.041404,0.004198,0.123132,0.000638,0.083005,0.026222
...,...,...,...,...,...,...,...,...,...,...,...,...
10089734,11504791,0.364706,0.004869,0.682274,0.0,0.103297,0.041404,0.004198,0.123132,0.000638,0.084166,0.026222
10089735,11504792,0.400000,0.055490,0.672241,0.0,0.139688,0.177725,0.004198,0.123132,0.000638,0.191811,0.249854
10089736,11504793,0.564706,0.050747,0.729097,0.0,0.139688,0.177725,0.240530,0.123132,0.228493,0.068065,0.207763
10089737,11504795,0.341176,0.004869,0.632107,0.0,0.103297,0.041404,0.004198,0.123132,0.000638,0.084166,0.026222


In [3]:
trueDF=pd.read_csv('trueDF.csv')
trueDF

,id,Age,Annual_Premium,Vintage,Response,Response_y,Response_y.1,Response_y.2,Response_y.3,Response_y.4,Response_y.5,Response_y.6
0,1,0.505882,0.109061,0.963211,1.0,0.139688,0.299049,0.24053,0.123132,0.228493,0.191811,0.207763
1,7,0.552941,0.075271,0.876254,1.0,0.103297,0.177725,0.24053,0.123132,0.228493,0.087718,0.207763
2,28,0.470588,0.004869,0.682274,1.0,0.139688,0.177725,0.24053,0.123132,0.228493,0.121274,0.260495
3,36,0.588235,0.054146,0.913043,1.0,0.103297,0.177725,0.24053,0.123132,0.228493,0.096163,0.192496
4,44,0.470588,0.067048,0.956522,1.0,0.139688,0.177725,0.24053,0.123132,0.228493,0.087718,0.192496
...,...,...,...,...,...,...,...,...,...,...,...,...
1415054,11504732,0.529412,0.060241,0.301003,1.0,0.139688,0.177725,0.24053,0.123132,0.228493,0.191811,0.192496
1415055,11504755,0.529412,0.069721,0.545151,1.0,0.139688,0.177725,0.24053,0.123132,0.228493,0.078031,0.111863
1415056,11504784,0.294118,0.089306,0.123746,1.0,0.139688,0.041404,0.24053,0.123132,0.228493,0.191811,0.056011
1415057,11504794,0.305882,0.054630,0.384615,1.0,0.103297,0.041404,0.24053,0.123132,0.228493,0.078031,0.026222


In [21]:
concatDF=pd.concat((trueDF,falseDF[:len(trueDF)]))
concatDF = concatDF.sample(frac=1)
concatDF = concatDF.reset_index(drop=True)

idDF=concatDF['id']
concatDF = concatDF.drop(columns=['id'])

responseDF=concatDF['Response']
concatDF = concatDF.drop(columns=['Response'])

concatDF

,Age,Annual_Premium,Vintage,Response_y,Response_y.1,Response_y.2,Response_y.3,Response_y.4,Response_y.5,Response_y.6
0,0.258824,0.095584,0.625418,0.139688,0.041404,0.240530,0.123132,0.228493,0.191811,0.026222
1,0.447059,0.086955,0.270903,0.139688,0.177725,0.240530,0.123132,0.228493,0.191811,0.192496
2,0.294118,0.063212,0.387960,0.139688,0.041404,0.004198,0.123132,0.000638,0.076968,0.026222
3,0.894118,0.093405,0.494983,0.139688,0.299049,0.240530,0.123132,0.228493,0.191811,0.207763
4,0.352941,0.072393,0.217391,0.103297,0.041404,0.240530,0.123132,0.228493,0.191811,0.171656
...,...,...,...,...,...,...,...,...,...,...
2830113,0.470588,0.052514,0.103679,0.103297,0.177725,0.240530,0.123132,0.228493,0.116748,0.192496
2830114,0.494118,0.004869,0.083612,0.139688,0.177725,0.004198,0.123132,0.000638,0.107787,0.192496
2830115,0.294118,0.043892,0.919732,0.103297,0.041404,0.004198,0.123132,0.000638,0.107787,0.026222
2830116,0.458824,0.095367,0.558528,0.139688,0.177725,0.240530,0.123132,0.228493,0.191811,0.207763


In [22]:
kf = KFold(n_splits=5)
kf.get_n_splits(concatDF.values)

for i, data in enumerate(kf.split(concatDF)):
    print(data)

(array([ 566024,  566025,  566026, ..., 2830115, 2830116, 2830117]), array([     0,      1,      2, ..., 566021, 566022, 566023]))
(array([      0,       1,       2, ..., 2830115, 2830116, 2830117]), array([ 566024,  566025,  566026, ..., 1132045, 1132046, 1132047]))
(array([      0,       1,       2, ..., 2830115, 2830116, 2830117]), array([1132048, 1132049, 1132050, ..., 1698069, 1698070, 1698071]))
(array([      0,       1,       2, ..., 2830115, 2830116, 2830117]), array([1698072, 1698073, 1698074, ..., 2264092, 2264093, 2264094]))
(array([      0,       1,       2, ..., 2264092, 2264093, 2264094]), array([2264095, 2264096, 2264097, ..., 2830115, 2830116, 2830117]))


In [6]:
clf = HistGradientBoostingClassifier(max_iter=100).fit(x_train, y_train)
clf.score(x_test, y_test)

0.8094596189939262